<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sorted_sparsity_selectivity_neuron_size_50_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

Training dataset size:  60000
Testing dataset size:  10000


In [ ]:
new_mnist_trainset =  [ [[],[]] for i in range(10)]
# new_mnist_testset  =  [ [[],[]] for i in range(10)]

for i in range(60000):
    for j in range(10):
        if mnist_trainset[i][1] == j:
            # image 
            new_mnist_trainset[j][0].append(mnist_trainset[i][0])  
            # label
            new_mnist_trainset[j][1].append(mnist_trainset[i][1])

# for i in range(10000):
#     for j in range(10):
#         if mnist_testset[i][1] == j:
#             # image 
#             new_mnist_testset[j][0].append(mnist_testset[i][0])  
#             # label
#             new_mnist_testset[j][1].append(mnist_testset[i][1])

image_trainset = list()
label_trainset = list()

# image_testset = list()
# label_testset = list()

for i in range(10):
    image_trainset.append(new_mnist_trainset[i][0])
    label_trainset.append(new_mnist_trainset[i][1])

# for i in range(10):
#     image_testset.append(new_mnist_testset[i][0])
#     label_testset.append(new_mnist_testset[i][1])

flattened_image_train = list()
flattened_label_train = list()

# flattened_image_test = list()
# flattened_label_test = list()

# flattening image 
for sublist in image_trainset:
    for val in sublist:
        flattened_image_train.append(val)

# flattening label
for sublist in label_trainset:
    for val in sublist:
        flattened_label_train.append(val)

# # flattening image 
# for sublist in image_testset:
#     for val in sublist:
#         flattened_image_test.append(val)

# # flattening label
# for sublist in label_testset:
#     for val in sublist:
#         flattened_label_test.append(val)

flattened_image_train = torch.stack(flattened_image_train)
flattened_label_train = torch.Tensor(flattened_label_train)
flattened_label_train = flattened_label_train.type(torch.LongTensor)

# flattened_image_test = torch.stack(flattened_image_test)
# flattened_label_test = torch.Tensor(flattened_label_test)
# flattened_label_test = flattened_label_test.type(torch.LongTensor)

train_dataset = TensorDataset(flattened_image_train, flattened_label_train)
train_dataloader = DataLoader(train_dataset, batch_size=50)

# test_dataset = TensorDataset(flattened_image_test, flattened_label_test)
test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, hidden_layer_neurons):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, hidden_layer_neurons)
        self.linear_2 = torch.nn.Linear(hidden_layer_neurons, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 30
def sparsity_selectivity_trainer(optimizer, model, hidden_layer_neurons):

    hidden_layer_each_neuron = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(hidden_layer_neurons)]
    hidden_layer_each_neuron = np.array(hidden_layer_each_neuron)

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    # ***************************** sparsity calc *****************************
    final_spareness = list()
    # ***************************** sparsity calc *****************************

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        # ***************************** sparsity calc *****************************
        hidden_layer_activation_list = list()
        # ***************************** sparsity calc *****************************

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************************** sparsity calc *****************************
            hidden_layer_activation_list.append(model.layer_activations)
            # ***************************** sparsity calc *****************************

            # find selectivity at the final epoch 
            if epoch == no_epochs - 1: # last epoch 
                for activation, label in zip(model.layer_activations, labels):
                    # shape of activation and label: 256 and 1 
                    
                    # get the actual value of item. This is because label is now Tensor 
                    label = label.item()

                    # this is not part of gradient calculcation 
                    with torch.no_grad():
                        activation = activation.numpy()

                    # for each image/label, append activation value of neuron 
                    for i in range(hidden_layer_neurons):    # number of neurons in hidden layer 
                        hidden_layer_each_neuron[i][label].append(activation[i])

        # ***************************** sparsity calc *****************************
        final_spareness.append(hidden_layer_activation_list)
        # ***************************** sparsity calc *****************************

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************************** selectivity calc *****************************
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(hidden_layer_neurons)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(hidden_layer_neurons)]

    # selectivity_list contains all of the selectivity of each neuron 
    selectivity_list = list()

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity_list
        selectivity_list.append(selectivity)
    # ***************************** selectivity calc *****************************

    # ***************************** sparsity calc *****************************
    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, hidden_layer_neurons))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)
     # ***************************** sparsity calc *****************************

    print("sparseness_list", sparseness_list)
    print("selectivity_list", selectivity_list)

    return test_acc, sparseness_list, selectivity_list

In [ ]:
hidden_layer_neurons1 = [64, 128, 256, 384, 512, 768, 1024]

# AdaDelta

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adadelta = Model(hidden_layer_neurons[i])
    print("model_Adadelta:", model_Adadelta)
    model_Adadelta.to(device)
    model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
    optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
    Adadelta_test_acc, Adadelta_sparsity_list, Adadelta_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_sparsity_selectivity_1_50_Adadelta.txt", "w")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparsity_list)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')
    else:
        f = open("neurons_sparsity_selectivity_1_50_Adadelta.txt", "a")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparsity_list)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')

f.close()

!cp neurons_sparsity_selectivity_1_50_Adadelta.txt /content/drive/MyDrive

Entering 0-th loop
model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)


# AdaGrad

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adagrad = Model(hidden_layer_neurons[i])
    print("model_Adagrad:", model_Adagrad)
    model_Adagrad.to(device)
    model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
    optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
    Adagrad_test_acc, Adagrad_sparsity_list, Adagrad_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_sparsity_selectivity_Adagrad_1_50.txt", "w")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparsity_list)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')
    else:
        f = open("neurons_sparsity_selectivity_Adagrad_1_50.txt", "a")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparsity_list)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')

f.close()

!cp neurons_sparsity_selectivity_Adagrad_1_50.txt /content/drive/MyDrive

# SGD


In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_SGD = Model(hidden_layer_neurons[i])
    print("model_SGD:", model_SGD)
    model_SGD.to(device)
    model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
    optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
    SGD_test_acc, SGD_sparsity_list, SGD_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_sparsity_selectivity_SGD_1_50.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparsity_list)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')
    else:
        f = open("neurons_sparsity_selectivity_SGD_1_50.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparsity_list)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')

f.close()

!cp neurons_sparsity_selectivity_SGD_1_50.txt /content/drive/MyDrive

# Adam

In [ ]:
for i in range(len(hidden_layer_neurons)):
    print(f"Entering {i}-th loop")
    model_Adam = Model(hidden_layer_neurons[i])
    print("model_Adam:", model_Adam)
    model_Adam.to(device)
    model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
    optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
    Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=hidden_layer_neurons[i])
    
    if i == 0:
        f = open("neurons_sparsity_selectivity_Adam_1_50.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparsity_list)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')
    else:
        f = open("neurons_sparsity_selectivity_Adam_1_50.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparsity_list)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')

f.close()

!cp neurons_sparsity_selectivity_Adam_1_50.txt /content/drive/MyDrive

In [ ]:
hidden_layer_neurons1 = [64, 128, 256, 384, 512, 768, 1024]

In [ ]:
model_Adam = Model(64)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=64)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.25198588, Test Loss: 0.15855411, Test Accuracy: 0.95370000

Epoch: 2/30, Train Loss: 0.12578949, Test Loss: 0.11986096, Test Accuracy: 0.96460000

Epoch: 3/30, Train Loss: 0.09790932, Test Loss: 0.10899269, Test Accuracy: 0.96650000

Epoch: 4/30, Train Loss: 0.08397265, Test Loss: 0.11409606, Test Accuracy: 0.96610000

Epoch: 5/30, Train Loss: 0.07209467, Test Loss: 0.10792961, Test Accuracy: 0.96870000

Epoch: 6/30, Train Loss: 0.06334574, Test Loss: 0.10678061, Test Accuracy: 0.97040000

Epoch: 7/30, Train Loss: 0.05699084, Test Loss: 0.11253214, Test Accuracy: 0.97020000

Epoch: 8/30, Train Loss: 0.05108855, Test Loss: 0.11307145, Test Accuracy: 0.97040000

Epoch: 9/30, Train Loss: 0.04738297, Test Loss: 0.11672325, Test Accuracy: 0.96930000

Epoch: 10/30, Train Loss: 0.04253052, Tes

In [ ]:
model_Adam = Model(128)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=128)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.22432642, Test Loss: 0.12577930, Test Accuracy: 0.96170000

Epoch: 2/30, Train Loss: 0.10123075, Test Loss: 0.10302111, Test Accuracy: 0.97040000

Epoch: 3/30, Train Loss: 0.07435209, Test Loss: 0.08469324, Test Accuracy: 0.97510000

Epoch: 4/30, Train Loss: 0.05626258, Test Loss: 0.10261932, Test Accuracy: 0.97170000

Epoch: 5/30, Train Loss: 0.04623749, Test Loss: 0.10283497, Test Accuracy: 0.97150000

Epoch: 6/30, Train Loss: 0.03917257, Test Loss: 0.08818581, Test Accuracy: 0.97790000

Epoch: 7/30, Train Loss: 0.03178251, Test Loss: 0.09448713, Test Accuracy: 0.97610000

Epoch: 8/30, Train Loss: 0.02650865, Test Loss: 0.11751689, Test Accuracy: 0.97140000

Epoch: 9/30, Train Loss: 0.02329846, Test Loss: 0.10171043, Test Accuracy: 0.97560000

Epoch: 10/30, Train Loss: 0.02044091, T

In [ ]:
model_Adam = Model(256)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=256)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.20718922, Test Loss: 0.10255038, Test Accuracy: 0.97040000

Epoch: 2/30, Train Loss: 0.08990572, Test Loss: 0.10265782, Test Accuracy: 0.96970000

Epoch: 3/30, Train Loss: 0.06350397, Test Loss: 0.08522690, Test Accuracy: 0.97690000

Epoch: 4/30, Train Loss: 0.04829945, Test Loss: 0.08215662, Test Accuracy: 0.97970000

Epoch: 5/30, Train Loss: 0.03661278, Test Loss: 0.09014102, Test Accuracy: 0.97880000

Epoch: 6/30, Train Loss: 0.03028339, Test Loss: 0.09336657, Test Accuracy: 0.97910000

Epoch: 7/30, Train Loss: 0.02300705, Test Loss: 0.11021034, Test Accuracy: 0.97670000

Epoch: 8/30, Train Loss: 0.01939068, Test Loss: 0.12636753, Test Accuracy: 0.97600000

Epoch: 9/30, Train Loss: 0.01669428, Test Loss: 0.09757675, Test Accuracy: 0.98010000

Epoch: 10/30, Train Loss: 0.01345836, T

In [ ]:
model_Adam = Model(384)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=384)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=384, bias=True)
  (linear_2): Linear(in_features=384, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.20096606, Test Loss: 0.10209418, Test Accuracy: 0.96890000

Epoch: 2/30, Train Loss: 0.08614406, Test Loss: 0.11023695, Test Accuracy: 0.96770000

Epoch: 3/30, Train Loss: 0.06218170, Test Loss: 0.07732103, Test Accuracy: 0.97890000

Epoch: 4/30, Train Loss: 0.04602217, Test Loss: 0.08442723, Test Accuracy: 0.97870000

Epoch: 5/30, Train Loss: 0.03598697, Test Loss: 0.09661062, Test Accuracy: 0.97920000

Epoch: 6/30, Train Loss: 0.02912243, Test Loss: 0.10361650, Test Accuracy: 0.97760000

Epoch: 7/30, Train Loss: 0.02413755, Test Loss: 0.10349518, Test Accuracy: 0.97970000

Epoch: 8/30, Train Loss: 0.02036208, Test Loss: 0.11466192, Test Accuracy: 0.97910000

Epoch: 9/30, Train Loss: 0.01604663, Test Loss: 0.11515065, Test Accuracy: 0.97910000

Epoch: 10/30, Train Loss: 0.01422882, T

In [ ]:
model_Adam = Model(512)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=512)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=512, bias=True)
  (linear_2): Linear(in_features=512, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.19906497, Test Loss: 0.11302352, Test Accuracy: 0.96540000

Epoch: 2/30, Train Loss: 0.08565973, Test Loss: 0.09198543, Test Accuracy: 0.97500000

Epoch: 3/30, Train Loss: 0.05830932, Test Loss: 0.09634455, Test Accuracy: 0.97700000

Epoch: 4/30, Train Loss: 0.04440080, Test Loss: 0.09184639, Test Accuracy: 0.97790000

Epoch: 5/30, Train Loss: 0.03453563, Test Loss: 0.09585245, Test Accuracy: 0.97790000

Epoch: 6/30, Train Loss: 0.02962860, Test Loss: 0.10369463, Test Accuracy: 0.98030000

Epoch: 7/30, Train Loss: 0.02314996, Test Loss: 0.10049772, Test Accuracy: 0.97990000

Epoch: 8/30, Train Loss: 0.02037488, Test Loss: 0.11393794, Test Accuracy: 0.98010000

Epoch: 9/30, Train Loss: 0.01888872, Test Loss: 0.10699455, Test Accuracy: 0.98060000

Epoch: 10/30, Train Loss: 0.01326043, T

In [ ]:
model_Adam = Model(768)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=768)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=768, bias=True)
  (linear_2): Linear(in_features=768, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.19656048, Test Loss: 0.10261850, Test Accuracy: 0.96920000

Epoch: 2/30, Train Loss: 0.08744990, Test Loss: 0.10333888, Test Accuracy: 0.97450000

Epoch: 3/30, Train Loss: 0.06103810, Test Loss: 0.08569655, Test Accuracy: 0.97990000

Epoch: 4/30, Train Loss: 0.04701021, Test Loss: 0.09513980, Test Accuracy: 0.97860000

Epoch: 5/30, Train Loss: 0.03527072, Test Loss: 0.09557252, Test Accuracy: 0.97840000

Epoch: 6/30, Train Loss: 0.02823984, Test Loss: 0.15494753, Test Accuracy: 0.96940000

Epoch: 7/30, Train Loss: 0.02761455, Test Loss: 0.11477742, Test Accuracy: 0.97950000

Epoch: 8/30, Train Loss: 0.02185533, Test Loss: 0.11348373, Test Accuracy: 0.98060000

Epoch: 9/30, Train Loss: 0.01844449, Test Loss: 0.12009893, Test Accuracy: 0.98060000

Epoch: 10/30, Train Loss: 0.01718136, T

In [ ]:
model_Adam = Model(1024)
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_sparsity_list, Adam_selectivity_list = sparsity_selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam, hidden_layer_neurons=1024)
print(Adam_test_acc)
print(Adam_sparsity_list)
print(np.average(Adam_selectivity_list))
print(np.std(Adam_selectivity_list))

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=1024, bias=True)
  (linear_2): Linear(in_features=1024, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/30, Train Loss: 0.19709336, Test Loss: 0.12178474, Test Accuracy: 0.96460000

Epoch: 2/30, Train Loss: 0.08684429, Test Loss: 0.12680174, Test Accuracy: 0.96420000

Epoch: 3/30, Train Loss: 0.06228071, Test Loss: 0.10610934, Test Accuracy: 0.97490000

Epoch: 4/30, Train Loss: 0.04630693, Test Loss: 0.11556572, Test Accuracy: 0.97460000

Epoch: 5/30, Train Loss: 0.03800436, Test Loss: 0.10572207, Test Accuracy: 0.97940000

Epoch: 6/30, Train Loss: 0.03112048, Test Loss: 0.14493706, Test Accuracy: 0.97490000

Epoch: 7/30, Train Loss: 0.02834638, Test Loss: 0.12663943, Test Accuracy: 0.97910000

Epoch: 8/30, Train Loss: 0.02383994, Test Loss: 0.13908864, Test Accuracy: 0.97850000

Epoch: 9/30, Train Loss: 0.01880494, Test Loss: 0.14244552, Test Accuracy: 0.97940000

Epoch: 10/30, Train Loss: 0.01714537,